In [ ]:
import os
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Tentukan jalur direktori untuk data latih, validasi, dan uji
train_data = "../Oprek/Ori_Data_Aug/train" 
val_data = "../Oprek/Ori_Data_Aug/val"
test_data = "../Oprek/Ori_Data_Aug/test"

# train_data = '../Oprek/data_split/train'
# test_data = '../Oprek/data_split/test'

In [ ]:
# Applying Data normalization and Data augmentation
img_datagen = ImageDataGenerator(
    # rescale=1./255
)

# validation_datagen = ImageDataGenerator(
#     # rescale=1./255
# )

train_generator = img_datagen.flow_from_directory(
    train_data,
    batch_size=32,
    color_mode="rgb",
    target_size=(224, 224),
    class_mode="categorical"
)

validation_generator = img_datagen.flow_from_directory(
    val_data,
    batch_size=32,
    color_mode="rgb",
    target_size=(224, 224),
    class_mode="categorical"
)

test_generator = img_datagen.flow_from_directory(
    test_data,
    batch_size=32,
    color_mode="rgb",
    target_size=(224, 224),
    class_mode="categorical"
)

# Model CNN Dengan Arsitekur Vgg16

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Membuat objek VGG16 tanpa lapisan terakhir (include_top=False)
base_model = VGG16(weights=None, include_top=False, input_shape=(224, 224, 3))

# Membuat model Sequential
model = Sequential()

# Menambahkan lapisan-lapisan VGG16 ke dalam model
model.add(base_model)

# Menambahkan lapisan-lapisan tambahan untuk klasifikasi
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(20, activation='softmax'))  

# Mengatur model untuk pembelajaran transfer dengan membekukan lapisan-lapisan VGG16
for layer in base_model.layers:
    layer.trainable = False

# Kompilasi model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Mencetak ringkasan arsitektur model
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define EarlyStopping callback
early_stopping_callback = EarlyStopping(
    monitor='accuracy', 
    patience=3, 
    min_delta=0.01, 
    baseline=0.98, 
    mode='max', 
    verbose=1
)

In [ ]:
# Train the model with EarlyStopping callback
history = model.fit(
    train_generator, 
    epochs=15, 
    validation_data=validation_generator,  
    callbacks=[early_stopping_callback]
)